In [10]:
from sklearn.ensemble import BaggingClassifier as bg
from sklearn.model_selection import train_test_split
import pandas as pd 
import xgboost as xgb 
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
#from sklearn import accuracy
import random
random.seed()
rand = random.randint(1,100)



In [11]:
laligadfmod = pd.read_pickle('laligadfmod')

y=laligadfmod['outcome']
X=laligadfmod.loc[:, laligadfmod.columns!='outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123, test_size=0.5, 
                                                    shuffle=True, stratify=y)

In [12]:
laligadf = pd.read_pickle('laligadfmod')
count = 0
temp1 = laligadf.reset_index()
temp = temp1['outcome'].copy()
for i in range(len(temp)):
    if(temp[i]==1):
        count = count+1
goals = count
misses = len(temp)-goals
weight = misses/goals
print(weight)

6.4858569051580695


In [13]:
param_grid = [{'n_estimators': range(5,10)}]
                

#boost = xgb.XGBClassifier(
#        n_estimators = 10,
#        learning_rate = .01,
#        scale_pos_weight = weight,
#        booster = 'gb_linear', use_label_encoder = False)

boost = xgb.XGBClassifier(objective ='reg:squarederror', use_label_encoder=False, scale_pos_weight = weight, learning_rate = .0001)
gs = GridSearchCV(estimator=boost, param_grid=param_grid, cv=10, scoring='accuracy')


gs.fit(X_train,y_train)
print('Best Accuracy: %.2f%%' % (gs.best_score_*100))
print('Best Params: %s' % gs.best_params_)
print('Test Accuracy: %.2f%%' % (gs.best_estimator_.score(X_test,y_test)*100))

Best Accuracy: 91.64%
Best Params: {'n_estimators': 6}
Test Accuracy: 92.13%


In [14]:

boost = xgb.XGBClassifier(n_estimators = 6, objective ='reg:squarederror', use_label_encoder=False, scale_pos_weight = weight, learning_rate = 0.0001)
pred = bg(base_estimator = boost,
        n_estimators = 500,
        random_state = rand
     ).fit(X_train,y_train)
pred.fit(X_train, y_train)

BaggingClassifier(base_estimator=XGBClassifier(base_score=None, booster=None,
                                               colsample_bylevel=None,
                                               colsample_bynode=None,
                                               colsample_bytree=None,
                                               enable_categorical=False,
                                               gamma=None, gpu_id=None,
                                               importance_type=None,
                                               interaction_constraints=None,
                                               learning_rate=0.0001,
                                               max_delta_step=None,
                                               max_depth=None,
                                               min_child_weight=None,
                                               missing=nan,
                                               monotone_constraints=None,
                

In [ ]:
print('Training Accuracy: %.2f%%' % (pred.score(X_train, y_train)*100))
print('Test Accuracy: %.2f%%' % (pred.score(X_test,y_test)*100))

Training Accuracy: 93.24%
Test Accuracy: 92.31%


In [ ]:
import numpy as np
from mlxtend.evaluate import mcnemar_table
